In [3]:
# Initialize notebook
import numpy as np
import json
import plotly.graph_objects as go
from bs4 import BeautifulSoup


METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

ENCOUNTER_DIFFICULTIES = {
    'Easy':   {
        'XP': [25,50,75,125,250,300,350,450,550,600,800,1000,1100,1250,1400,1600,2000,2100,2400,2800], 
        'color': '#1F77B4'
    },
    'Medium': {
        'XP': [50,100,150,250,500,600,750,900,1100,1200,1600,2000,2200,2500,2800,3200,3900,4200,4900,5700], 
        'color': '#FF7F0E'
    },
    'Hard':   {
        'XP': [75,150,225,375,750,900,1100,1400,1600,1900,2400,3000,3400,3800,4300,4800,5900,6300,7300,8500], 
        'color': '#2CA02C'
    },
    'Deadly': {
        'XP': [100,200,400,500,1100,1400,1700,2100,2400,2800,3600,4500,5100,5700,6400,7200,8800,9500,10900,12700], 
        'color': '#D62728'
    },
    'Daily':  {
        'XP': [300,600,1200,1700,3500,4000,5000,6000,7500,9000,10500,11500,13500,15000,18000,20000,25000,27000,30000,40000], 
        'color': '#9467BD'
    },
}

In [4]:
# Setup template and html config for plotly figures.

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        legend=dict(
            font_size=10,
            tracegroupgap=0,
            orientation='h',
        )
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': False, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [3]:
# Plots XP thresholds for each encounter difficulty. Thresholds are normalized to half daily XP budget.

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter XP / half Daily XP',
            range=[0, 1.5],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
        )
    )
)

levels = np.array(range(1,21,1))
enc_XP_budget = 0.5*np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])

for diff in ['Easy','Medium','Hard','Deadly']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    rel_XP = np.array(d['XP'])/enc_XP_budget
    rel_XP_mean = np.median(rel_XP)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=rel_XP,
        mode='lines', 
        line=dict(color=d['color'], dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff,
        hovertemplate=
            '<b>' + diff + '</b><br>'+
            'lvl %{x}<br>'+
            'ratio %{y:.2f}'+
            '<extra></extra>'
    ))
    fig.add_trace(go.Scatter(
        x=levels, 
        y=20*[rel_XP_mean],
        mode='lines', 
        line=dict(color=d['color'], dash='dash'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff,
        showlegend=False,
        hoverinfo='skip',
    ))

ref_XP = 0.5*np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='half Daily XP',
    showlegend=False,
    hoverinfo='skip'
))


# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-encounter-xp-thresholds-vs-level-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-encounter-xp-thresholds-vs-level-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [5]:
# Plots average encounter XP for PCs, calculated for a full adventuring day with Medium encounters and two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated encounter XP / half Daily XP',
            range=[0, 1.5],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
        )
    )
)

enc_XP_budget = 0.5*np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
for diff in ['Medium']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'./pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)
    
    eXP = np.zeros(20)
    for pc_class in PC_CLASSES:
        eXP += np.array([x['encounter XP mean'] for x in pc_data[pc_class]])
    eXP /= len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/enc_XP_budget,
        mode='lines+markers', 
        line=dict(color='red', dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name='mean encounter XP',
        hovertemplate=
            '<b>encounter XP</b><br>'+
            'lvl %{x}<br>'+
            'ratio %{y:.2f}'+
            '<extra></extra>'
    ))

ref_XP = 0.5*np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='half Daily XP',
    showlegend=False,
    hoverinfo='skip'
))

ref_XP = np.array(ENCOUNTER_DIFFICULTIES['Deadly']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dashdot'),
    name='Deadly XP',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-half-daily-xp-vs-level-medium-adventuring-days-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-half-daily-xp-vs-level-medium-adventuring-days-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [6]:
# Plots average daily XP for PCs, calculated for a full adventuring day with Medium encounters and two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated daily XP / Daily XP',
            range=[0, 1.5],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
        )
    )
)

daily_XP_budget = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
for diff in ['Medium']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'./pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    eXP = np.zeros(20)
    for pc_class in PC_CLASSES:
        eXP += np.array([x['XP mean'] for x in pc_data[pc_class]])
    eXP /= len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/daily_XP_budget,
        mode='lines+markers', 
        line=dict(color='red', dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name='mean daily XP',
        hovertemplate=
            '<b>daily XP</b><br>'+
            'lvl %{x}<br>'+
            'ratio %{y:.2f}'+
            '<extra></extra>'
    ))

ref_XP = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/daily_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='Daily XP',
    showlegend=False,
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-full-daily-xp-vs-level-medium-adventuring-days-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-full-daily-xp-vs-level-medium-adventuring-days-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [7]:
# Plots eDPR for each class, calculated for a full adventuring day with Medium encounters and two short rests.
encounter_difficulty = 'Medium'
pc_data_file = f'./pc data - {encounter_difficulty} 2 short rests.json'

with open(pc_data_file, 'r') as fin:
    pc_data = json.load(fin)

levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='effective damage per round',
            range=[0, 110],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

eDPRmean = np.zeros(20)
for pc in PC_CLASSES:
    eDPR = np.array([x['effective damage per round mean'] for x in pc_data[pc]])
    eDPRmean += eDPR

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eDPR,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'eDPR %{y:.1f}'+
            '<extra></extra>'
    ))

eDPRmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eDPRmean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save small format figure
if SAVEFIGS:
    file_name = f'./fig-pcs-encounter-edpr-vs-level-{encounter_difficulty.lower()}-adventuring-days-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-pcs-encounter-edpr-vs-level-{encounter_difficulty.lower()}-adventuring-days-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [8]:
# Plots eHP for each class, calculated for a full adventuring day with Medium encounters and two short rests.
encounter_difficulty = 'Medium'
pc_data_file = f'./pc data - {encounter_difficulty} 2 short rests.json'

with open(pc_data_file, 'r') as fin:
    pc_data = json.load(fin)

levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='effective hit points',
            range=[0, 800],
            tick0=0, dtick=100,
            minor=dict(tick0=0, dtick=50),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

eHPmean = np.zeros(20)
for pc in PC_CLASSES:
    eXP  = np.array([x['encounter XP mean'] for x in pc_data[pc]])
    eDPR = np.array([x['effective damage per round mean'] for x in pc_data[pc]])
    eHP = eXP/eDPR
    #eHP = np.array([x['encounter effective hit points mean'] for x in pc_data[pc]])
    eHPmean += eHP
    fig.add_trace(go.Scatter(
        x=levels, 
        y=eHP,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'eHP %{y:.1f}'+
            '<extra></extra>'
    ))

eHPmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eHPmean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pcs-encounter-ehp-vs-level-{encounter_difficulty.lower()}-adventuring-days-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-pcs-encounter-ehp-vs-level-{encounter_difficulty.lower()}-adventuring-days-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [9]:
# Plots encounter XP for each class, calculated for a full adventuring day with Medium encounters and two short rests.
encounter_difficulty = 'Medium'
pc_data_file = f'./pc data - {encounter_difficulty} 2 short rests.json'

with open(pc_data_file, 'r') as fin:
    pc_data = json.load(fin)

levels = np.array(range(1,21,1))
enc_XP_budget = 0.5 * np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter XP / half Daily XP',
            range=[0.0, 3.5],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

eXPmean = np.zeros(20)
for pc in PC_CLASSES:
    eXP  = np.array([x['encounter XP mean'] for x in pc_data[pc]])
    eXPmean += eXP
    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/enc_XP_budget,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'enc XP %{y:.2f}'+
            '<extra></extra>'
    ))

eXPmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eXPmean/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))


# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pcs-encounter-xp-vs-level-{encounter_difficulty.lower()}-adventuring-days-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-pcs-encounter-xp-vs-level-{encounter_difficulty.lower()}-adventuring-days-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))